# Consultas propuestas por el enunciado

#### Imports y definición de funciones

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import warnings
# suprimir future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
spark = SparkSession.builder \
    .appName("ConsultasEnunciado") \
    .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

### 1. ¿Cuál es el Estado que más descuentos tiene en total? ¿y en promedio?

#### Limpieza y preparación de columnas

In [ ]:
orders = sqlContext.read.csv(
    'data/orders.csv',
    header=True, inferSchema=True
)

#### Resolución

### 2. ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado 'Refunded'? 
###    ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

#### Limpieza y preparación de columnas

#### Resolución

### 3. Para cada tipo de pago y segmento de cliente, <br>devolver la suma y el promedio expresado como porcentaje <br>de clientes activos y de consentimiento de marketing

#### Resolución

### 4. Para los productos que contienen en su descripción la palabra "stuff", <br> calcular el peso total de su inventario agrupado por marca, <br> mostrar sólo la marca y el peso total de las 5 más pesadas

#### Limpieza y preparación de columnas

#### Resolución

### 5. Calcular el porcentaje de productos cuyo stock es al menos 20% <br> más alto que el stock promedio de su marca

#### Limpieza y preparación de columnas

#### Resolución

### 6. Obtener la cantidad de órdenes que no hayan comprado ninguno de los 10 productos más vendidos

#### Limpieza y preparación de columnas

#### Resolución